In [1]:
%pip install git+https://github.com/aldanor/ipybind.git -qqq

Note: you may need to restart the kernel to use updated packages.


In [1]:
%load_ext ipybind

In [2]:
from math import sqrt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

# Datos

In [128]:
!wget -q -O datos_retocados.csv "https://www.dropbox.com/scl/fi/xnztguety6brdy7t2lfge/wiki_movie_plots_deduped_sample.csv?rlkey=7m867bh7ruilw66qlh7ozl9g4&dl=1"

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
tabla = pd.read_csv("datos_retocados.csv")

In [17]:
generos_dict = {"western": 0, "science fiction": 1, "romance": 2, "crime": 3}
generos_list = ["western", "science fiction", "romance", "crime"]

In [18]:
tokens = np.hstack(tabla["tokens"].apply(lambda x: x.split()).values)
unique_tokens = pd.Series(tokens).value_counts().index[:1000].values
unique_tokens_dict = dict(zip(unique_tokens, range(len(unique_tokens))))

tabla_matriz = np.zeros((len(tabla), len(unique_tokens)), dtype=int)
for i, row in tabla.iterrows():
    for token in row["tokens"].split():
        if unique_tokens_dict.get(token,False)!=False:
            tabla_matriz[i, unique_tokens_dict[token]] += 1

In [127]:
train_tabla = tabla.loc[tabla["split"] == "train"].to_numpy()
train_generos = train_tabla[:, 5]
train_generos = [generos_dict[x] for x in train_generos]
train_vectores = np.zeros((len(train_generos), len(unique_tokens)), dtype=int)
for i in range(len(train_tabla)):
    for token in train_tabla[i, 8].split():
        if unique_tokens_dict.get(token,False) != False:
            train_vectores[i, unique_tokens_dict[token]] += 1

# Horrible, preguntar como hacer
tempa = []
tempb = []
for i in range(int(len(train_generos)/4)):
    for j in range(4):
        tempa.append(train_generos[i + j*int(len(train_generos)/4)])
        tempb.append(train_vectores[i + j*int(len(train_vectores)/4)])
train_generos = np.array(tempa)
train_vectores = np.array(tempb)


# KNN

In [108]:
def productoPunto(vectorA, vectorB):
    producto = 0
    for i in range(len(vectorA)) : 
       producto = vectorA[i]*vectorB[i] + producto
    return producto 

def norma(vector):
    valor = 0
    for i in range(len(vector)): 
        valor = vector[i]**2 + valor
    valor = sqrt(valor)
    return valor    

def distanciaCoseno(vectorA, vectorB):
    distancia = 1 - (productoPunto(vectorA,vectorB) / (norma(vectorA) * norma(vectorB)))
    return distancia

In [109]:
def KVecinos(vector, modelo_generos, modelo_vectores, k):
    vecinos = []
    for i in range(len(modelo_vectores)): 
        vecinos.append((modelo_generos[i], distanciaCoseno(modelo_vectores[i], vector)))
      
    vecinos = sorted(vecinos, key=lambda tup: tup[1])
    kvecinos = []
    for i in range(k):
        kvecinos.append(vecinos[i][0])
    return kvecinos

def KNN(vector, modelo_generos, modelo_vectores, k):
    aparicionesDeClase = [0,0,0,0]

    kvecinos = KVecinos(vector, modelo_generos, modelo_vectores, k)

    for vecino in kvecinos:
        aparicionesDeClase[vecino] = aparicionesDeClase[vecino] + 1
    
    maxIndice = 0
    maxApariciones = 0       
    for i in range(len(aparicionesDeClase)):
        if(aparicionesDeClase[i] > maxApariciones):
            maxIndice = i
            maxApariciones = aparicionesDeClase[i]
    return generos_list[maxIndice]

In [110]:
modelo_generos = train_generos[19:]
modelo_vectores = train_vectores[19:]
for i in range(19):
    vector = train_vectores[i]
    genero_esperado = generos_list[train_generos[i]]
    print("Esperado: " + genero_esperado + " Resultado: " + KNN(vector, modelo_generos, modelo_vectores, 10))

In [111]:
def normaNumpy(vector):
    return np.linalg.norm(vector)

def distanciaCosenoNumpy(vectorA, vectorB):
    return 1 - (np.dot(vectorA, vectorB) / (normaNumpy(vectorA) * normaNumpy(vectorB)))

In [112]:
def KVecinosNumpy(vector, modelo_generos, modelo_vectores, k):
    vecinos = np.zeros((len(modelo_generos), 2))
    vecinos[:, 0] = modelo_generos
    vecinos[:, 1] = [distanciaCosenoNumpy(vector, x) for x in modelo_vectores]
        
    vecinos = vecinos[vecinos[:, 1].argsort()]
    kvecinos = np.array(vecinos[:k, 0], dtype=int)
    return kvecinos


def KNNNumpy(vector, modelo_generos, modelo_vectores, k):
    kvecinos = KVecinosNumpy(vector, modelo_generos, modelo_vectores, k)
    return scipy.stats.mode(kvecinos).mode

In [113]:
modelo_generos = train_generos[19:]
modelo_vectores = train_vectores[19:]
for i in range(19):
    vector = train_vectores[i]
    genero_esperado = generos_list[train_generos[i]]
    print("Esperado: " + genero_esperado + " Resultado: " + generos_list[KNNNumpy(vector, modelo_generos, modelo_vectores, 10)])

# Método de la potencia

In [62]:
%%pybind11

#include <iostream>
#include <pybind11/numpy.h>
#include <eigen3/Eigen/Dense>
#include <pybind11/eigen.h>
#include <math.h>

using namespace std;
using namespace Eigen;

PYBIND11_PLUGIN(metodo_potencia) {
    py::module m("metodo_potencia");
    m.def("calcular_autovector", [](const MatrixXd& matriz) {
        VectorXd autovec = VectorXd::Random(matriz.cols()) * 10;
        cout << autovec << endl;
    });
    return m.ptr();
}

In [67]:
A = np.array([[7., 2., 3.],
              [0., 2., 0.],
              [-6., -2., -2.]], dtype=np.float64)
calcular_autovector(A)

9.41268
8.04416
 7.0184


# K-Fold

In [114]:
def kfold(generos, vectores, particiones, vecinos):
    correctos = 0
    tamano_particion = int(len(generos) / particiones)
    for i in range(particiones):
        azul_start = i * tamano_particion
        azul_end = i * tamano_particion + tamano_particion
        modelo_generos = np.concatenate((generos[:azul_start], generos[azul_end:]))
        modelo_vectores = np.concatenate((vectores[:azul_start], vectores[azul_end:]))
        for j in range(azul_start, azul_end):
            vector = vectores[j]
            genero_esperado = generos[j]
            genero_res = KNNNumpy(vector, modelo_generos, modelo_vectores, vecinos)
            if genero_esperado == genero_res:
                correctos = correctos + 1
    return correctos / len(generos)

In [126]:
kfold(train_generos, train_vectores, 9, 5)